# Daily and Monthly statistics

This notebook will demonstrate how to use the earthkit libraries to access some ERA5 data, reduce the date to daily (or monthly) time steps and plot the data.

For this exercise we will use the earthkit-data package to access the data, earthkit-climate to calculate the daily(/mothly) statistics and earthkit-maps to plot the results.

The earthkit-climate routines are *currently* based on xarray, hence they return raw xarray objects. *This may change in future versions of Earthkit*.

In [3]:
import numpy as np # Everyone loves a numpy!

import earthkit as ek
from earthkit.climate import aggregate as ek_aggregate


## Request some data from the CDS

For this example we are going to use the first 3 months of 2015

In [4]:
cds_dataset_name = 'reanalysis-era5-single-levels'

# We use an Earthkit bounding box object to describe our area,
# this clears up any lack of clarity of the order of: North, South, East, West.
area = ek_data.utils.bbox.BoundingBox(north=80, south=20, west=-30, east=100)
cds_request = {
        'product_type': 'reanalysis',
        'variable': '2m_temperature',
        'year': '2015',
        'month': [
            '01', '02', '03',
            # '04', '05', '06',
            # '07', '08', '09',
            # '10', '11', '12',
        ],
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31',
        ],
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
}
era5_T2M_data = ek_data.from_source("cds", cds_dataset_name, cds_request)

cds_request.update({"variable": "total_precipitation"})
era5_TP_data = ek_data.from_source("cds", cds_dataset_name, cds_request)


View the data object in the format that you prefer:

In [5]:
# As an xarray:
era5_T2M_data.to_xarray()
# # As a fieldlist
# era5_data.ls()
# # As a numpy array:
# era5_data.to_numpy()

<xarray.Dataset>
Dimensions:     (number: 1, time: 2160, step: 1, surface: 1, latitude: 721,
                 longitude: 1440)
Coordinates:
  * number      (number) int64 0
  * time        (time) datetime64[ns] 2015-01-01 ... 2015-03-31T23:00:00
  * step        (step) timedelta64[ns] 00:00:00
  * surface     (surface) float64 0.0
  * latitude    (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    valid_time  (time, step) datetime64[ns] ...
Data variables:
    t2m         (number, time, step, surface, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-09-06T16:22 GRIB to CDM+CF via cfgrib-0.9.1...

## Calculate the daily statistics

### Daily mean

First we calculate the daily mean of the 2m air temperature

In [6]:
era5_daily_mean = ek_aggregate.temporal.daily_mean(era5_T2M_data)
era5_daily_mean

<xarray.Dataset>
Dimensions:    (number: 1, step: 1, surface: 1, latitude: 721, longitude: 1440,
                time: 90)
Coordinates:
  * number     (number) int64 0
  * step       (step) timedelta64[ns] 00:00:00
  * surface    (surface) float64 0.0
  * latitude   (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * time       (time) datetime64[ns] 2015-01-01 2015-01-02 ... 2015-03-31
Data variables:
    t2m        (time, number, step, surface, latitude, longitude) float32 246...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-09-06T16:22 GRIB to CDM+CF via cfgrib-0.9.1...

### Different time zone

ERA5 data is provided with a time dimension local the UTC, hence the daily statistics are generally only
useful to locations in that time zone. It is possible to provide a time shift if you are more interested in
a time zone away from UTC.

The following calculates the daily mean relative to a timezone 12 hours ahead of UTC. 
Please be aware that your data request should probably be modified to account for the
additional timesteps required for the start/end days.
In the example below we have one additional day in our returned object because the first
and last days are now made up of partial days and should be omitted from further analysis.

In [7]:
era5_daily_mean_12 = ek_aggregate.temporal.daily_mean(era5_T2M_data, time_shift={'hours': 12})
era5_daily_mean_12

<xarray.Dataset>
Dimensions:    (number: 1, step: 1, surface: 1, latitude: 721, longitude: 1440,
                time: 91)
Coordinates:
  * number     (number) int64 0
  * step       (step) timedelta64[ns] 00:00:00
  * surface    (surface) float64 0.0
  * latitude   (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * time       (time) datetime64[ns] 2015-01-01 2015-01-02 ... 2015-04-01
Data variables:
    t2m        (time, number, step, surface, latitude, longitude) float32 246...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-09-06T16:22 GRIB to CDM+CF via cfgrib-0.9.1...

In [8]:
daily_accumulation = ek_aggregate.temporal.daily_sum(era5_TP_data)
daily_accumulation